In [96]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [17]:
loansData=pd.read_csv('https://github.com/Thinkful-Ed/curric-data-001-data-sets/raw/master/loans/loansData.csv')
loansData.dropna(inplace=True)
loansData['Interest.Rate']=loansData['Interest.Rate'].str.strip('%').astype(float)
loansData['Yearly.Income']=loansData['Monthly.Income']*12
loansData['Home_Coded']=pd.Categorical(loansData['Home.Ownership']).codes

In [24]:
loansData[['Home.Ownership','Home_Coded']]

,Home.Ownership,Home_Coded
81174,MORTGAGE,0
99592,MORTGAGE,0
80059,MORTGAGE,0
15825,MORTGAGE,0
33182,RENT,3


In [107]:
#Univariate Model
#int_rate = f(yearly.income)
model=smf.ols(formula='loansData["Interest.Rate"] ~ loansData["Yearly.Income"]',data=loansData)
results=model.fit()

print(results.summary())

                                OLS Regression Results                                
Dep. Variable:     loansData["Interest.Rate"]   R-squared:                       0.000
Model:                                    OLS   Adj. R-squared:                 -0.000
Method:                         Least Squares   F-statistic:                    0.4168
Date:                        Fri, 15 Jul 2016   Prob (F-statistic):              0.519
Time:                                16:55:52   Log-Likelihood:                -7115.5
No. Observations:                        2498   AIC:                         1.424e+04
Df Residuals:                            2496   BIC:                         1.425e+04
Df Model:                                   1                                         
Covariance Type:                    nonrobust                                         
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------

In [108]:
#Multivariate Model
#int_rate = f(yearly.income)
model=smf.ols(formula='loansData["Interest.Rate"] ~ loansData["Yearly.Income"] + loansData["Home_Coded"]',data=loansData)
results=model.fit()

print(results.summary())

                                OLS Regression Results                                
Dep. Variable:     loansData["Interest.Rate"]   R-squared:                       0.007
Model:                                    OLS   Adj. R-squared:                  0.006
Method:                         Least Squares   F-statistic:                     8.274
Date:                        Fri, 15 Jul 2016   Prob (F-statistic):           0.000262
Time:                                16:57:37   Log-Likelihood:                -7107.5
No. Observations:                        2498   AIC:                         1.422e+04
Df Residuals:                            2495   BIC:                         1.424e+04
Df Model:                                   2                                         
Covariance Type:                    nonrobust                                         
                                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------

In [109]:
#Multivariate Model
#int_rate = f(yearly.income, home_coded, yearly.income * home_coded)

model=smf.ols(formula='loansData["Interest.Rate"] ~ loansData["Yearly.Income"] * loansData["Home_Coded"]',data=loansData)
results=model.fit()

print(results.summary())

                                OLS Regression Results                                
Dep. Variable:     loansData["Interest.Rate"]   R-squared:                       0.008
Model:                                    OLS   Adj. R-squared:                  0.007
Method:                         Least Squares   F-statistic:                     6.588
Date:                        Fri, 15 Jul 2016   Prob (F-statistic):           0.000197
Time:                                16:58:25   Log-Likelihood:                -7105.9
No. Observations:                        2498   AIC:                         1.422e+04
Df Residuals:                            2494   BIC:                         1.424e+04
Df Model:                                   3                                         
Covariance Type:                    nonrobust                                         
                                                         coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------